In [36]:
# Import API Key
from gptapi_keys import gpt_key, org_key

# Import Dependencies
import os
import openai
openai.organization = org_key
openai.api_key = gpt_key

In [37]:
openai.Model.list()

<OpenAIObject list at 0x1e0037ef880> JSON: {
  "object": "list",
  "data": [
    {
      "id": "whisper-1",
      "object": "model",
      "created": 1677532384,
      "owned_by": "openai-internal",
      "permission": [
        {
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "object": "model_permission",
          "created": 1683912666,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "whisper-1",
      "parent": null
    },
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "object": "model_permission",
          "created": 166908